In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [ ]:
# data
# http://data.beta.nyc/dataset/unofficial-mta-transit-data-archive

# documentation 
# http://bustime.mta.info/wiki/Developers/ArchiveData

#gtfs spec
# https://developers.google.com/transit/gtfs/reference
!wget -nc http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150128.csv.xz
!7z -y x bus_time_20150128.csv.xz
!wget -nc http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150129.csv.xz
!7z -y x bus_time_20150129.csv.xz

!wget -nc http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip
!unzip -o gtfs_nyct_bus_20150103.zip

In [2]:
#ETL
import pandas as pd
import numpy as np
from datetime import timedelta, datetime


def convert_to_int64(row):
    try:
        return np.int64(row)
    except ValueError:
        return np.nan

def utc_to_est(row):
    try:
        return row - timedelta(hours=5)
    except:
        raise

csv_f = pd.read_csv
archive = pd.concat([csv_f("bus_time_20150128.csv"),
                     csv_f("bus_time_20150129.csv")])

trips = pd.read_csv("trips.txt")
stops = pd.read_csv("stops.txt")
schedules = pd.read_csv("stop_times.txt")
schedules.departure_time = pd.to_datetime(schedules.departure_time)
archive.timestamp = pd.to_datetime(archive.timestamp).apply(utc_to_est)
archive.next_stop_id = archive.next_stop_id.apply(convert_to_int64)

/opt/conda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
#query and clean
live_archive = archive[archive.block_assigned > 0] # "assigned" to a route
today = live_archive[live_archive.service_date == 20150128] # due to time shift we get some dates in 1/27 and 1/29
#throw away trips with <15 reports
good_trips_only = today.groupby(today.trip_id).filter(lambda group: len(group) > 15)

#write this as our "clean" dataset
good_trips_only.to_csv("realtime.csv")

#merged dataframe with both 
partial = good_trips_only.merge(trips, on='trip_id') 
df = partial.merge(stops, left_on="next_stop_id", right_on="stop_id")